In [1]:
import sys
import os 

sys.path.append(os.path.dirname(os.getcwd()))
from Augur.rag import GraphRAG
from Augur.agent_templates import StanzaTaggingAgent, GenerativeTaggingAgent
import pandas as pd

df_train = pd.read_json("../datafiles/train-data.json")

ontology_files = ['../datafiles/dbpedia_2016-10.owl']
model = "sentence-transformers/all-mpnet-base-v2"
ontology_db = GraphRAG(model, ontology_files)                 

In [2]:
from rdflib import Graph

dbpedia_graph = Graph().parse('../datafiles/dbpedia_2016-10.owl', format='xml')
dbpedia_ids = {element for triple in dbpedia_graph for element in triple if 'http://dbpedia.org/' in element}

In [3]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
agent = StanzaTaggingAgent(model_id)

2024-03-13 10:59:12 INFO: Downloading default packages for language: en (English) ...
2024-03-13 10:59:13 INFO: File exists: /root/stanza_resources/en/default.zip
2024-03-13 10:59:17 INFO: Finished downloading models and saved to /root/stanza_resources.
2024-03-13 10:59:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-13 10:59:18 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2024-03-13 10:59:18 INFO: Using device: cuda
2024-03-13 10:59:18 INFO: Loading: tokenize
2024-03-13 10:59:18 INFO: Loading: pos
2024-03-13 10:59:18 INFO: Loading: lemma
2024-03-13 10:59:18 INFO: Loading: constituency
2024-03-13 10:59:19 INFO: Loading: depparse
2024-03-13 10:59:19 INFO: Loading: sentiment
2024-03-13 10:59:19 INFO: Loading: ner
2024-03-13 10:59:20 INFO: Done loading processors!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
input_text = agent(df_train.corrected_question[20])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Write a rdfs:comment suitable for the meaning of this dbpedia ontology identifier: built. Be succint. [/INST] "Built: Identifier for structures or objects that have been created or produced by human activity."</s>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Write a rdfs:comment suitable for the meaning of this dbpedia ontology identifier: was rented. Be succint. [/INST] "This identifier represents the property of an object being rented out or having been rented in the past."</s>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Write a rdfs:comment suitable for the meaning of this dbpedia ontology identifier: Copa Centroamericana. Be succint. [/INST] "The Copa Centroamericana is a football tournament for national teams from the Central American region, organized by the Confederation of Central American Football (UNCAF). It is held every even year and serves as the qualification event for the CONCACAF Gold Cup."</s>


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Write a rdfs:comment suitable for the meaning of this dbpedia ontology identifier: stadium. Be succint. [/INST] A stadium is a large venue for outdoor sports and other events, with a fixed seating capacity for spectators. (rdfs:comment type="Stadium";)</s>


In [8]:
df_train.corrected_question[20]

'Who built the stadium which was rented for the 2013 Copa Centroamericana ?'

In [5]:
query = list(input_text.values())
output = ontology_db.process_query(query, 5)
graph  = Graph().parse(data=output, format='turtle')

In [9]:
print(graph.serialize(format='turtle'))

@prefix : <http://dbpedia.org/ontology/> .
@prefix d0: <http://www.ontologydesignpatterns.org/ont/d0.owl#> .
@prefix dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix wikidata: <http://www.wikidata.org/entity/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:AmericanFootballLeague a owl:Class ;
    rdfs:label "american football league"@en ;
    rdfs:comment "A group of sports teams that compete against each other in american football."@en ;
    rdfs:subClassOf :SportsLeague ;
    prov:wasDerivedFrom <http://mappings.dbpedia.org/index.php/OntologyClass:AmericanFootballLeague> .

:Arena a owl:Class ;
    rdfs:label "arena"@en ;
    rdfs:comment "An arena 

In [7]:
{str(s) for s,_,_ in graph}

{'http://dbpedia.org/ontology/Activity',
 'http://dbpedia.org/ontology/Agent',
 'http://dbpedia.org/ontology/AmericanFootballLeague',
 'http://dbpedia.org/ontology/ArchitecturalStructure',
 'http://dbpedia.org/ontology/Arena',
 'http://dbpedia.org/ontology/Athlete',
 'http://dbpedia.org/ontology/BaseballLeague',
 'http://dbpedia.org/ontology/Bridge',
 'http://dbpedia.org/ontology/Building',
 'http://dbpedia.org/ontology/CanadianFootballLeague',
 'http://dbpedia.org/ontology/CareerStation',
 'http://dbpedia.org/ontology/Company',
 'http://dbpedia.org/ontology/Country',
 'http://dbpedia.org/ontology/FieldHockeyLeague',
 'http://dbpedia.org/ontology/FootballMatch',
 'http://dbpedia.org/ontology/HandballLeague',
 'http://dbpedia.org/ontology/Hotel',
 'http://dbpedia.org/ontology/List',
 'http://dbpedia.org/ontology/Memorial',
 'http://dbpedia.org/ontology/Monument',
 'http://dbpedia.org/ontology/Museum',
 'http://dbpedia.org/ontology/Organisation',
 'http://dbpedia.org/ontology/Park',
 'ht

In [8]:
gold_standard = pd.read_csv('../datafiles/lcquad_test_gold.csv')

In [9]:
gold_standard.sparql_query[0]

' SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Marine_Corps_Air_Station_Kaneohe_Bay> <http://dbpedia.org/property/architect> ?uri. <http://dbpedia.org/resource/New_Sanno_Hotel> <http://dbpedia.org/ontology/tenant> ?uri} '

In [10]:
import re

pattern = r'<(http[s]?://[^ >]*dbpedia\.org[^ >]*)>'
#pattern = r'<(http[s]?://[^ >]*dbpedia\.org/resource/(?!([A-Z]|%[0-9A-F]{2}))[^ >]*)>' # no resources from database


In [11]:
gold_standard['identifiers'] = gold_standard.sparql_query.apply( lambda x : set(re.findall(pattern, x)))

In [12]:
gold_standard['identifiers']

0      {http://dbpedia.org/ontology/tenant, http://db...
1      {http://dbpedia.org/resource/Muslim_Brotherhoo...
2      {http://dbpedia.org/property/developer, http:/...
3      {http://dbpedia.org/resource/Eric_Schiller, ht...
4      {http://dbpedia.org/ontology/industry, http://...
                             ...                        
995    {http://dbpedia.org/resource/Judaism, http://d...
996    {http://dbpedia.org/resource/American_Hockey_L...
997    {http://dbpedia.org/resource/Easy_Street_(film...
998    {http://dbpedia.org/property/school, http://db...
999    {http://dbpedia.org/resource/Richard_Coke, htt...
Name: identifiers, Length: 1000, dtype: object

In [48]:
for s,p,o in dbpedia_graph:
    if 'resource' in p:
        print(o)

In [26]:
gold_standard.output[0]

"{'head': {'link': [], 'vars': ['uri']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/United_States_Navy'}}]}}"

In [51]:
gold_standard.sparql_query[1]

'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/international> <http://dbpedia.org/resource/Muslim_Brotherhood> . ?x <http://dbpedia.org/ontology/religion> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/PoliticalParty>}'